<h2 align="center"> Data Mining and Machine Learning </h2>
<h3 align="center"> Final Project </h3>
<h2 align="center"> <b> <i> CrashSpot </i> </b> </h2>
<h4 align="center"> Lorenzo Ceccanti matr. 564490 </h4>

### <b> Geografical Clustering with DBSCAN</b>

We start considering the dataset in which the granularity is per accident in order to perform a geographical clustering.

In [89]:
import os
import pandas as pd
import numpy as np
from pyclustertend import hopkins
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

R_EARTH_KM = 6371.0

df = pd.read_csv(os.path.join('../editedDataset', 'CLEANED_brasilEnglishAggr.csv'))

In [90]:
df.loc[:,"city"].value_counts().head()

city
BRASILIA     6612
CURITIBA     6472
SAO JOSE     5104
GUARULHOS    4936
PALHOCA      4164
Name: count, dtype: int64

In [91]:
# The theory suggests us to sample uniformly the dataset picking up a number of points lower lower than the total number of instances
# we have. We do so by using the sample method on the DataFrame df contained our cleaned dataset
def evaluate_clustering_tendency(df):
    # We pick only latitude and longitude attributes as they are the one on which the clustering activity focuses
    X = df[['latitude', 'longitude']].copy()
    dim = X.shape[0]
    if dim < 50:
        n_samples = dim
    else:
        n_samples = 50
    # The conversion is necessary since hopkins() expects an ndarray as first parameter
    H = hopkins(X.to_numpy(), n_samples)
    return H

In [92]:
import plotly.express as px
from kneed import KneeLocator
def plot_k_dist_graph(X,k, cause):

    # Converting in radians
    coords_rad = np.radians(X[['latitude', 'longitude']].to_numpy())

    # For each object, we consider its distance with the k-th nearest neighbor
    neigh = NearestNeighbors(n_neighbors=k, metric="haversine")
    neigh.fit(coords_rad)
    # The kneighbors method of neighb object returns the distance as the first
    # parameter and the index of the nearest point as second parameter
    # However, we are not interested in the second parameter
    distances_rad, _ = neigh.kneighbors(coords_rad)
    # the k-1 column in distances is the distance with the k-th nearest neighbor
    # : is to consider the distance between EACH point and that k-th neighbor
    # We consider a decreasing order, that is not implemented in numpy

    kth_rad = distances_rad[:,k-1]
    kth_km = kth_rad * R_EARTH_KM

    kth_km_sorted = np.sort(kth_km)[::-1]

    # Looking for the knee of the K-distance graph
    x = np.arange(1, len(kth_km_sorted)+1)
    y = kth_km_sorted
    knee = KneeLocator(x, y, curve="convex", direction="decreasing")
    eps = y[knee.knee] if knee.knee is not None else None
    
    distances_df = pd.DataFrame({
        # Here I'm creating the indexes for the x-axis (the number of points)
        "Points": range(len(kth_km_sorted)),
        f"{k}-th nearest neighbor distance": kth_km_sorted
    })
    fig = px.line(distances_df, x = 'Points', y = f'{k}-th nearest neighbor distance', title = f'K-distance Graph: {cause}')
    fig.show()
    return eps

In [93]:
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score
def get_GeoDBSCAN_metrics(eps_km, minPts, gpsCoords):
    # Converting in radians
    coords = np.radians(gpsCoords).to_numpy()

    # Fit
    dbscan = DBSCAN(eps = eps_km/R_EARTH_KM, min_samples=minPts, metric='haversine')
    dbscan.fit(coords)

    labels = dbscan.labels_
    # Preparing a mask to exclude the indexes belonging to outliers
    mask = labels != -1

    # Here we have only the data which are not considered as outliers by the DBSCAN
    data_filtered = coords[mask]
    labels_filtered = labels[mask]
    
    # First metric: Percentage of core points and number of outliers
    count_core_points = np.sum(mask)
    total_points = np.size(mask)
    count_outliers = total_points - count_core_points

    # Second metric: Number of clusters
    number_of_clusters = len(set(dbscan.labels_[dbscan.labels_ >= 0]))
    if number_of_clusters <= 1:
        return "No clusters", ""

    # Third metric: Davies-Bouldin Index
    davies_bouldin_index = davies_bouldin_score(data_filtered, labels_filtered)

    # Fourth metric: Silouette Coefficent
    sil_coefficent = metrics.silhouette_score(data_filtered, labels_filtered, metric='haversine')

    # Fifth metrics: Calinski-Harabasz index
    calinski_index = metrics.calinski_harabasz_score(data_filtered, labels_filtered)

    # I return a dictionary containing the metrics associated with the Eps and MinPts chosen
    df_results = {
        'eps_km': eps_km,
        'min_samples': minPts,
        'count_core_points': count_core_points,
        'count_outliers': count_outliers,
        'corePoints_outliers_ratio': count_core_points/count_outliers,
        'number_of_clusters': number_of_clusters,
        'davies_bouldin_index': davies_bouldin_index,
        'sil_coefficent': sil_coefficent,
        'calinski_index': calinski_index
    }
    return df_results, labels    

In [94]:
def dbscan_tuning(minEps, maxEps, stepEps, minPtsArr, gpsCoords):
    eps_to_test = []
    for eps in np.arange(minEps, maxEps, stepEps):
        eps_to_test.append(eps)

    # It's an array of dictionaries
    arr_tuning_results = []

    for eps in eps_to_test:
        for minPts in minPtsArr:
            df_metric, _ = get_GeoDBSCAN_metrics(eps, minPts, gpsCoords)
            if isinstance(df_metric, str) == False:
                arr_tuning_results.append(df_metric)

    df_tuning_results = pd.DataFrame(arr_tuning_results)
    return df_tuning_results

In [95]:
def get_dbscan_tuning_df(cause, minEps, maxEps, stepEps, minPtsArr, data):
    df_filtered = data.query(f"general_cause_of_accident == '{cause}'").copy()
    minEps_, maxEps_ = min(minEps, maxEps), max(minEps, maxEps)

    gpsCoords = df_filtered[['latitude', 'longitude']].copy()
    tuning_results = dbscan_tuning(minEps_, maxEps_, stepEps, minPtsArr, gpsCoords)
    if tuning_results.empty:
        return "No clusters"
    else:
        tuning_results = tuning_results.query("corePoints_outliers_ratio > 1.4").sort_values(
                by=["corePoints_outliers_ratio", "sil_coefficent", "davies_bouldin_index", "calinski_index"],
                ascending=[True, False, True, False]
        )
        return tuning_results.query("corePoints_outliers_ratio > 1.4")

In [ ]:
import plotly.express as px
import pandas as pd

def plot_map2(data: pd.DataFrame, text: str, clusters_stats):

    cont_scale = [
            [0.0,  "green"],
            [0.1,  "green"],
            [0.2,  "green"],
            [0.3,  "yellow"],
            [0.4,  "yellow"],
            [0.5,  "yellow"],
            [0.6,  "red"],
            [0.7,  "red"],
            [0.8,  "red"],
            [0.9,  "red"],
            [1.0,  "red"]
        ]
    
    fig = px.scatter_map(
        data,
        lat="latitude",
        lon="longitude",
        color = data["labels"].map(clusters_stats.set_index("labels")["hotspot_score"]),
        color_continuous_scale=cont_scale
    )
    fig.update_layout(
        mapbox_style="carto-positron",
        title=f"Map plot ({text})"
    )
    fig.show()

In [97]:
import plotly.express as px
import pandas as pd

def plot_map(data: pd.DataFrame, text: str, color_col: str, zoom_factor=3):
    if color_col == "labels":
        # Variabile numerica/continua
        cont_scale = [
            [0.0,  "gray"],
            [0.1,  "cyan"],
            [0.2,  "lightblue"],
            [0.3,  "blue"],
            [0.4,  "green"],
            [0.5,  "lightgreen"],
            [0.6,  "yellow"],
            [0.7,  "gold"],
            [0.8,  "orange"],
            [0.9,  "orangered"],
            [1.0,  "red"]
        ]
        fig = px.scatter_map(
            data,
            lat="latitude",
            lon="longitude",
            color=color_col,  # nome colonna
            color_continuous_scale=cont_scale,
            hover_data=["inverse_data", color_col]
        )
    else:
        # Variabile categorica (testuale)
        discrete_map = {
            "Without victims": "green",
            "With injured victims": "orange",
            "With dead victims": "red"
        }
        fig = px.scatter_map(
            data,
            lat="latitude",
            lon="longitude",
            color=color_col,     # nome colonna
            color_discrete_map=discrete_map,
            category_orders={color_col: ["Without victims", "With injured victims", "With dead victims"]},
            hover_data=["inverse_data", color_col]
        )

    fig.update_layout(
        mapbox_style="carto-positron",
        title=f"Map plot ({text})"
    )
    fig.show()


In [98]:
from sklearn.preprocessing import MinMaxScaler
def launch(param, query, cause_to_analyze, minPtsUser, epsUser):
    """ Launch the clustering proces
    Args:
        param: Could be City or State
        query: The name of the city/state
    """
    if param == "City":
        # We select a specific city
        df_selection = df.copy().query(f"city == '{query}'")
    if param == "State":
        # We select a specific state
        df_selection = df.copy().query(f"state == '{query}'")
    
    # We map specific causes into more general causes
    cause_mapping = {
        "Brake slam": ["Abrupt use of the car's brake"],
        "Minor traffic offense": ["Absence of sinalization",
                                  "Disobedience to laws of transit by the pedestrian",
                                  "car's on sidewalk"],
        "Traffic offense": ["Driver broke the laws of transit", "Irregular access",
                            "Lane change maneuver",
                            "Stopping at a prohibited place",
                            "The driver passed the next car improperly",
                            "Traffic with a motorcycle (or similar) between lanes",
                            "Acessing the road without seeing the presence of other vehicles"],
        "Major traffic offense": ["Disrespecting the intersection", 
                                  "Driver changed the lane illegally",
                                  "Driver disrespected the red traffic light",
                                  "Driver was in the opposite direction",
                                  "Driving on the breakdown lane",
                                  "Prohibited conversion"],
        "Driver distraction": ["Driver using cellphone",
                             "Driver was sleeping",
                             "Driver's lack of reaction",
                             "Driver's lack of attention to conveyance"],
        "Road defect":  ["Inadequate sinalization of the road",
                         "Curvy road", "No breakdown lanes", "Other flaws/problems in the road",
                         "Poor ilumination (of the road)",
                         "Road's defect",
                         "Roads with holes without cement",
                         "Sinking or ondulation in the pavement",
                         "Slippery track",
                         "Uneven breakdown lane",
                         "Unlevel track",
                         "Urban area without appropriate pedestrian walking"],
        "Road condition": ["Accumulation of water on the road", "Fog",
                           "Natural phenomena",
                           "Obstacle in the road",
                           "Oil accumulation on the road",
                           "Rain",
                           "Road had lots of sand/wreckage",
                           "Road works (in maintenance)",
                           "Static object on the drainage gate",
                           "Visibility restriction"],
        "Alcohol": ["Alcohol and/or drug ingestion by the pedestrian", "Alcohol consumption",
                    "Alcohol ingestion by the driver"],
        "Drugs": ["Driver was using drugs"],
        "Driver behavior": ["External fight"],
        "Animals": ["Animals on the road"],
        "Veichle not human fault": ["Car's brake problem", 
                           "Car's suspension system with problems", 
                           "Deficiency of vehicle's sinalization/ilumination system",
                           "Electrical or mechanical flaws",
                           "Mechanical loss/defect of vehicle"],
        "Veichle human fault": ["Excessive load/cargo", "Excessive use of the car's tire"],
        "Driver health": ["Cardiac attack", "Driver had a cardiac attack"],
        "Safe distance": ["Disrespect of safe distance from the next car",
                          "Driver failed to keep distance from the vehicle in front"],
        "High speed": ["Incompatible velocity"],
        "Pedestrian involved": ["Pedestrian was crossing the road outside of the crosswalk",
                                "Pedestrian was walking in the road",
                                "Pedestrian's lack of attention",
                                "Unexpected pedestrian entry"]
    }

    # Since for Pandas it's more convenient to have the specific causes as key, we reverse the mapping of the dictionary
    reverse_mapping = {specific: general 
                    for general, specifics in cause_mapping.items() 
                    for specific in specifics}
    df_selection["general_cause_of_accident"] = df_selection["cause_of_accident"].map(reverse_mapping)

    for cause in cause_to_analyze:
        
        df_filtered = df_selection.query(f"general_cause_of_accident == '{cause}'").copy()
        hopkins_val = evaluate_clustering_tendency(df_filtered)
        minpts = 4
        eps = plot_k_dist_graph(df_filtered, k=minpts, cause=cause)
        print(f'Hopkins {hopkins_val}')
        print(f'Elbow: {eps}')

        df_filtered = df_selection.query(f"general_cause_of_accident == '{cause}'").copy()
        gpsCoords = df_filtered[['latitude', 'longitude']].copy()

        # Plotting original data
        print("Plotting original data:")
        plot_map(data = df_filtered, text = f'{param}: {query}, cause: {cause} - original data', color_col='victims_condition', zoom_factor=3)
        print(f"DBSCAN tuning, {param}: {query}, cause: {cause}")
        tuning_df = get_dbscan_tuning_df(cause=cause, minEps = 0.3, maxEps = 2, stepEps=0.1, minPtsArr=[2,3,4], data=df_filtered).query("corePoints_outliers_ratio > 1.4")
        # Applying labels
        algorithm_stats, labels = get_GeoDBSCAN_metrics(eps_km=epsUser, minPts=minPtsUser, gpsCoords=gpsCoords)
        df_filtered = df_filtered.copy()
        df_filtered['labels'] = labels

        # Preparing the second plot and the hotspot score
        data_to_plot = df_filtered[df_filtered['labels'] > -1].copy()
        # Adding a rank for victims_condition
        victims_condition_map = {
            "Without victims": 0,
            "With injured victims": 1,
            "With dead victims": 2
        }
        data_to_plot['victims_condition_rank'] = data_to_plot["victims_condition"].map(victims_condition_map)

        # Plot showing the clustering labels only
        print("Plot according to clustering labels")
        plot_map(data = data_to_plot, text = f'{param}: {query} - {cause} - cluster labels - DBSCAN', color_col='labels')  # outliers are excluded

        # Computing the hotspot score
        group = data_to_plot.groupby("labels")
        accidents_per_label = group.size().reset_index(name="accidents_per_label") # it's a series
        sum_victims_condition_rank = group["victims_condition_rank"].sum().reset_index(name="sum_victims_condition_rank") 
        clusters_stats = pd.merge(accidents_per_label, sum_victims_condition_rank, on="labels")

        alpha = 0.2
        beta = 0.8
        clusters_stats['hotspot_score'] = alpha*clusters_stats['accidents_per_label'] + beta*clusters_stats['sum_victims_condition_rank']

        scaler = MinMaxScaler()
        clusters_stats["hotspot_score"] = scaler.fit_transform(clusters_stats[["hotspot_score"]])

        # Second plot
        print("Plot showing the hotspots:")
        plot_map2(data = data_to_plot, text = f'{param}: {query} - {cause} - hotspots - DBSCAN', clusters_stats=clusters_stats)  # outliers are excluded

        return tuning_df

In [102]:
# cause_to_analyze = ['High speed', 'Alcohol', 'Animals', 'Major traffic offense', 'Pedestrian involved']
# cause_to_analyze = ['High speed', 'Alcohol', 'Animals']
cause_to_analyze = ['High speed']
min_pts = 4
eps = 0.5
tuning_df = launch('City', 'BRASILIA', cause_to_analyze, min_pts, eps)

Hopkins 0.025997914374086883
Elbow: 1.9801956246486574
Plotting original data:


DBSCAN tuning, City: BRASILIA, cause: High speed
Plot according to clustering labels


Plot showing the hotspots:


In [100]:
tuning_df

,eps_km,min_samples,count_core_points,count_outliers,corePoints_outliers_ratio,number_of_clusters,davies_bouldin_index,sil_coefficent,calinski_index
2,0.3,4,309,176,1.755682,46,0.131143,0.909561,508115.822741
5,0.4,4,344,141,2.439716,46,0.204221,0.825942,83877.216527
8,0.5,4,356,129,2.759690,42,0.239934,0.746137,42705.877125
1,0.3,3,362,123,2.943089,63,0.136346,0.899409,379227.239568
4,0.4,3,384,101,3.801980,58,0.201286,0.815605,70897.480968
11,0.6,4,386,99,3.898990,41,0.304349,0.663107,26234.543753
14,0.7,4,392,93,4.215054,35,0.336075,0.637883,15998.521901
7,0.5,3,393,92,4.271739,54,0.235264,0.744041,39282.426802
17,0.8,4,398,87,4.574713,32,0.355291,0.581620,11667.754283
20,0.9,4,400,85,4.705882,25,0.355878,0.607538,9391.316483


In [106]:
cause_to_analyze = ['High speed']
min_pts = 4
eps = 0.4
tuning_df = launch('City', 'CURITIBA', cause_to_analyze, min_pts, eps)

Hopkins 0.0011526760892790048
Elbow: 1.7431966676387116
Plotting original data:


DBSCAN tuning, City: CURITIBA, cause: High speed
Plot according to clustering labels


Plot showing the hotspots:


In [104]:
tuning_df

,eps_km,min_samples,count_core_points,count_outliers,corePoints_outliers_ratio,number_of_clusters,davies_bouldin_index,sil_coefficent,calinski_index
2,0.3,4,474,103,4.601942,49,0.334483,0.690805,3655.521639
5,0.4,4,498,79,6.303797,40,0.435705,0.550265,1637.815987
1,0.3,3,504,73,6.904110,57,0.367048,0.622220,1490.846680
8,0.5,4,519,58,8.948276,33,0.420907,0.426285,1090.628926
11,0.6,4,524,53,9.886792,19,0.486748,0.305478,482.485066
4,0.4,3,526,51,10.313725,47,0.428493,0.520815,1464.699769
7,0.5,3,538,39,13.794872,38,0.403776,0.421477,1010.598308
14,0.7,4,541,36,15.027778,15,0.535069,0.308952,409.621566
10,0.6,3,544,33,16.484848,25,0.443426,0.293069,393.934127
0,0.3,2,546,31,17.612903,78,0.347915,0.609957,1285.894677


In [109]:
cause_to_analyze = ['High speed']
min_pts = 3
eps = 0.3
tuning_df = launch('State', 'RJ', cause_to_analyze, min_pts, eps)

Hopkins 0.008782870060175656
Elbow: 4.705531357169097
Plotting original data:


DBSCAN tuning, State: RJ, cause: High speed
Plot according to clustering labels


Plot showing the hotspots:


In [108]:
tuning_df

,eps_km,min_samples,count_core_points,count_outliers,corePoints_outliers_ratio,number_of_clusters,davies_bouldin_index,sil_coefficent,calinski_index
1,0.3,3,1475,1005,1.467662,265,0.153484,0.839845,911015.149886
11,0.6,4,1533,947,1.618796,166,0.253726,0.687944,126601.784243
4,0.4,3,1597,883,1.808607,266,0.186601,0.777292,315875.693124
14,0.7,4,1659,821,2.020706,161,0.275403,0.618562,54447.003573
7,0.5,3,1710,770,2.220779,261,0.212635,0.737279,192242.657930
17,0.8,4,1783,697,2.558106,156,0.300370,0.556467,21009.813536
10,0.6,3,1797,683,2.631040,244,0.258707,0.670878,109995.423054
13,0.7,3,1868,612,3.052288,226,0.272598,0.622369,52618.620095
20,0.9,4,1912,568,3.366197,146,0.311699,0.550624,16973.553451
0,0.3,2,1927,553,3.484629,491,0.151725,0.837684,854561.852563
